<p style="text-align: center;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/World_Health_Organization_Logo.svg/375px-World_Health_Organization_Logo.svg.png"
         alt="WHO Logo"
         style="display: block; margin: auto; height: 50px;" />
</p>

# 🌍 World Health Organisation Project: Predicting Life Expectancy 📊


<p style="text-align: left;">
    <img src="https://th.bing.com/th/id/OIP.-lAhNnbsu_F7YmCUiStBdAHaDt?w=256&h=180&c=7&r=0&o=5&dpr=1.3&pid=1.7 https://th.bing.com/th/id/OIP.-lAhNnbsu_F7YmCUiStBdAHaDt?w=256&h=180&c=7&r=0&o=5&dpr=1.3&pid=1.7"
         alt="Kung Fu Panda"
         style="display: block; margin: auto; height: 50px;" />
</p>

```
Kung-Fu Pandas!
Jamie, Kat, Pedro, Moka
```


## Overview 🧐
We are working on two predictive models for life expectancy:
1. **Detailed & Accurate Model** 📈
2. **Ethical Model** 💡


---

## Process & Steps Taken 🛠️



### **Importing the Dataset & Preparation** ⬇️
Loading the data and getting it ready for analysis.

In [ ]:
# Importing libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Use the train-test-split functionality from sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Use statsmodels for both the model & its evaluation
import statsmodels.api as sm # Where we'll get the model from
import statsmodels.tools     # Get the evaluation metrics
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Life Expectancy Data.csv to Life Expectancy Data.csv


In [ ]:
# Read the CSV file into a DataFrame for analysis

df = pd.read_csv('Life Expectancy Data.csv')
df.shape

(2864, 21)

---

### 1. **Train Test Splitting** 🔀
**Preparing for train test splitting:**
* Creating our features and targets sets

In [ ]:
## We are going to make a list of all of our columns, and then remove price from that list - since we don't want to include the price in X

feature_cols = list(df.columns) # The columns that we'll use as features
feature_cols.remove('Life_expectancy') # The price is the target, not a feature! Hence its removal

In [ ]:
X = df[feature_cols] # Declaring our features
y = df['Life_expectancy'] # Setting our target


---

**Creating our __4 quadrants__:**
* Both _training_ sets: `X_train`, `y_train`
* Both _testing_ sets: `X_test`, `y_test`

In [ ]:
# Creating our 4 quadrants --> both training and both testing sets

X_train, X_test, y_train, y_test = train_test_split(X, # Features set
                                                   y, # Target set
                                                   test_size=0.2, # % of data allocated to testing --> going with the standard 80-20 split
                                                   random_state=43) # Adding a random state for sample reproducibility

In [ ]:
X_train.head()

,Country,Region,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,BMI,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing
125,Papua New Guinea,Oceania,2014,41.2,52.4,227.2180,1.35,67,65,25.4,43,73,0.40,2565,7.95,1.3,1.3,4.3,0,1
648,Indonesia,Asia,2012,25.9,31.2,181.1225,0.08,83,78,22.7,87,83,0.20,2981,248.45,1.5,1.4,7.6,0,1
2729,Barbados,Central America and Caribbean,2000,13.7,15.1,127.4130,7.31,93,41,26.5,86,93,0.61,16277,0.27,4.3,4.2,9.0,0,1
1406,Lesotho,Africa,2010,70.9,96.4,548.4965,2.75,93,81,24.7,92,93,12.44,996,2.00,7.2,7.0,5.6,0,1
1634,Sri Lanka,Asia,2003,13.0,15.1,137.4770,1.69,86,90,21.8,98,99,0.02,2037,19.22,15.4,15.5,10.2,0,1


>Now that we've effectively split our training and testing sets, we conduct __sanity checks__ to ensure they have been created properly with <u>no apparent errors</u>

In [ ]:
# Checking the indices match between both training and both testing sets

print(all(X_train.index == y_train.index))
print(all(X_test.index == y_test.index))

# Checking the number of observations match between both training and both testing sets

print(X_train.shape[0] == y_train.shape[0])
print(X_test.shape[0] == y_test.shape[0])

True
True
True
True


### 2. **Feature Engineering**

>* Defining a function for feature engineering to be applied to our training sets
>* Applying the function, creating our feature engineered training set, and storing it in __`X_train_fe`__

In [ ]:
def feature_eng(df):
    df_fe = df.copy()
    df_fe.insert(loc=0, column='const', value=1)
    df_fe_scale = df_fe[['GDP_per_capita', 'Population_mln', 'Adult_mortality', 'Infant_deaths', 'BMI', 'Under_five_deaths']] # Scaling the numerical columns to make our model more robust
    rob = RobustScaler() ## Initialise scaler
    rob.fit(df_fe_scale) ## Fit the data

    ## Transform the data according to the scaler
    ## Save it as a new dataframe called df_fe_scale_rob
    df_fe_scale_rob = rob.transform(df_fe_scale)
    df_fe[['GDP_per_capita', 'Population_mln', 'Adult_mortality', 'Infant_deaths', 'BMI', 'Under_five_deaths']] = df_fe_scale_rob # Effectively applying the scaling to the columns in our dataframe
    return df_fe # Returning the feature engineered version of our df

In [ ]:
X_train_fe = feature_eng(X_train)

>Looking at __`X_train_fe`__ and making sure all changes have been correctly executed:
>* Creating a constant column
>* Scaling numerical ones

In [ ]:
X_train_fe.head()

,const,Country,Region,Year,Infant_deaths,Under_five_deaths,Adult_mortality,Alcohol_consumption,Hepatitis_B,Measles,...,Polio,Diphtheria,Incidents_HIV,GDP_per_capita,Population_mln,Thinness_ten_nineteen_years,Thinness_five_nine_years,Schooling,Economy_status_Developed,Economy_status_Developing
125,1,Papua New Guinea,Oceania,2014,0.551461,0.525847,0.456962,1.35,67,65,...,43,73,0.40,-0.155859,0.005297,1.3,1.3,4.3,0,1
648,1,Indonesia,Asia,2012,0.162643,0.147950,0.123047,0.08,83,78,...,87,83,0.20,-0.116420,11.587286,1.5,1.4,7.6,0,1
2729,1,Barbados,Central America and Caribbean,2000,-0.147395,-0.139037,-0.266025,7.31,93,41,...,86,93,0.61,1.144103,-0.364556,4.3,4.2,9.0,0,1
1406,1,Lesotho,Africa,2010,1.306226,1.310160,2.784304,2.75,93,81,...,92,93,12.44,-0.304608,-0.281242,7.2,7.0,5.6,0,1
1634,1,Sri Lanka,Asia,2003,-0.165184,-0.139037,-0.193121,1.69,86,90,...,98,99,0.02,-0.205916,0.548038,15.4,15.5,10.2,0,1


In [ ]:
## ORIGINAL FEATURES SET WHICH LED TO GOOD RESULTS! Keeping it on the back burner to not lose the set that got us to a good place

# feature_cols = ['const', 'Infant_deaths', 'Adult_mortality', 'Economy_status_Developed', 'GDP_per_capita', 'Schooling', 'BMI', 'Incidents_HIV']

In [ ]:
## SECOND SET OF FEATURES WHICH LED TO LOWER RMSE'S AND A LESS OVERFIT MODEL! Keeping them on the back burner as we experiment with other sets of features

# feature_cols = ['const', 'Infant_deaths', 'Under_five_deaths', 'Adult_mortality', 'Economy_status_Developed', 'Schooling', 'BMI', 'Incidents_HIV', 'Thinness_ten_nineteen_years']

In [ ]:
# Final, definitive set of features we'll use for our robust model --> After having experimented with Ridge and Lasso Regression as well as VIF

feature_cols = ['const', 'Infant_deaths', 'Under_five_deaths', 'Adult_mortality', 'Economy_status_Developed', 'Schooling', 'BMI', 'Incidents_HIV', 'Thinness_ten_nineteen_years']

### 3. **Fitting our Model and Analysing the Summary** 🧮
>Passing our DataFrames into the `sm.OLS()` method
>
>Fitting our model and storing it in __results__

In [ ]:
lin_reg = sm.OLS(y_train, X_train_fe[feature_cols])
results = lin_reg.fit()

In [ ]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Life_expectancy   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 1.231e+04
Date:                Mon, 07 Apr 2025   Prob (F-statistic):               0.00
Time:                        10:12:28   Log-Likelihood:                -4026.4
No. Observations:                2291   AIC:                             8071.
Df Residuals:                    2282   BIC:                             8123.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          70.3811      0.165    427.347      0.000      70.058      70.704
Infant_deaths                  -2.5090      0.269     -9.319      0.000      -3.037      -1.981
Under_five_deaths              -2.5013      0.233    -10.737      0.000      -2.958      -2.044
Adult_mortality                -6.8781      0.090    -76.145      0.000      -7.055      -6.701
Economy_status_Developed        1.3142      0.102     12.862      0.000       1.114       1.515
Schooling                       0.1561      0.018      8.502      0.000       0.120       0.192
BMI                            -0.4354      0.068     -6.410      0.000      -0.569      -0.302
Incidents_HIV                   0.1425      0.021      6.713      0.000       0.101       0.184
Thinness_ten_nineteen_years    -0.0371      0.009     -4.068      0.000      -0.055      -0.019
==============================================================================
Omnibus:                       10.995   Durbin-Watson:                   1.986
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               11.125
Skew:                           0.151   Prob(JB):                      0.00384
Kurtosis:                       3.159   Cond. No.                         113.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

---

### 4. **Experimenting with VIF --> Can we improve our results?** ⚙️



#### !Using code from the in-class LinReg part 4 notebook!

In [ ]:
def calculate_vif(X, thresh = 5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        # this bit uses list comprehension to gather all the VIF values of the different variables
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif)) # getting the index of the highest VIF value
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc] # we delete the highest VIF value on condition that it's higher than the threshold
            dropped = True # if we deleted anything, we set the 'dropped' value to True to stay in the while loop

    print('Remaining variables:')
    print(X.columns[variables]) # finally, we print the variables that are still in our set
    return X.iloc[:, variables] # and return our X cut down to the remaining variables

In [ ]:
vif_features = calculate_vif(X_train_fe[feature_cols])

dropping 'Infant_deaths' at index: 1
dropping 'const' at index: 0
dropping 'Adult_mortality' at index: 1
Remaining variables:
Index(['Under_five_deaths', 'Economy_status_Developed', 'Schooling', 'BMI',
       'Incidents_HIV', 'Thinness_ten_nineteen_years'],
      dtype='object')


In [ ]:
vif_lin_reg = sm.OLS(y_train, vif_features)
vif_results = vif_lin_reg.fit()

In [ ]:
vif_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:        Life_expectancy   R-squared (uncentered):                   0.963
Model:                            OLS   Adj. R-squared (uncentered):              0.963
Method:                 Least Squares   F-statistic:                              9933.
Date:                Mon, 07 Apr 2025   Prob (F-statistic):                        0.00
Time:                        10:13:30   Log-Likelihood:                         -9189.1
No. Observations:                2291   AIC:                                  1.839e+04
Df Residuals:                    2285   BIC:                                  1.842e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Under_five_deaths               9.5076      0.515     18.452      0.000       8.497      10.518
Economy_status_Developed       -6.0217      0.888     -6.781      0.000      -7.763      -4.280
Schooling                       7.4965      0.063    118.353      0.000       7.372       7.621
BMI                            -1.0823      0.637     -1.699      0.089      -2.331       0.167
Incidents_HIV                  -2.0216      0.136    -14.861      0.000      -2.288      -1.755
Thinness_ten_nineteen_years     1.7838      0.077     23.115      0.000       1.632       1.935
==============================================================================
Omnibus:                        7.584   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.023   Jarque-Bera (JB):                7.361
Skew:                          -0.113   Prob(JB):                       0.0252
Kurtosis:                       2.839   Cond. No.                         31.5
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**Notes on VIF:**

>After experimenting with VIF's suggestion for features, we came to the conclusion that the model turned out to be worse than what we'd come to previously:
>This is because:
>* The R-squared __decreased__
>* AIC and BIC __became much larger__
>* The __`.summary()`__ method gave us a warning about R-squared being computed without centering --> as it is __missing the constant__
>
>With these new results in mind, we decided to hold on to the features we'd used before as we believe they had led to a more robust model

---

### 5. **Creating Predictions --> Training Set** 🔑
>Using the `.predict()` method, we effectively create our predictions based on our features __training__ set
>
>We store results inside __y_pred__

In [ ]:
y_pred = results.predict(X_train_fe[feature_cols])

>To analyse our model's performance, we resorted to the RMSE metric
>
>* This is because RMSE effectively penalises outliers in the data
>    * Even though we've scaled our data, we still want to account for the different values we have across our numerical columns

In [ ]:
train_rmse = sm.tools.eval_measures.rmse(y_train, y_pred)

print(train_rmse)

1.402932764410591


---

### 6. **Creating Predictions --> Testing Set** 🔑
>We now run the same process but for our features __testing__ set

In [ ]:
X_test_fe = feature_eng(X_test)

In [ ]:
y_pred_test = results.predict(X_test_fe[feature_cols])

In [ ]:
test_rmse = sm.tools.eval_measures.rmse(y_test, y_pred_test)

print(test_rmse)

1.412967124402819


>To get a better feel for the difference in RMSE's from training to testing, we also calculated the __percentage difference between the two__;
>
>* This will effectively tell us __how much the testing RMSE increased when compared to the training one__

In [ ]:
pct_diff = ((test_rmse/train_rmse)-1)*100

In [ ]:
print(f'Percentage difference between testing and training: {round(pct_diff, 1)}%')

Percentage difference between testing and training: 0.7%


>As the model is only <u>slightly overfit</u> (~0.01 or 0.7% diff between training and testing), and we have managed to __hit the baseline target of a sub-2 RMSE__ whilst <u>maintaining good metrics all around</u> (__0.977 R-squared__, the __lowest AIC and BIC__ we achieved after experimenting with different sets of feature_cols, __113 condition number__), we believe we've built a __robust and effective model__!

---